## Script responsável por gerar embeddings com Universal Sentence Encoder (USE)

O **Universal Sentence Encoder (USE)** é um modelo desenvolvido pelo Google, projetado para gerar embeddings semânticos de sentenças e textos. Diferentemente de métodos tradicionais que utilizam vetores palavra a palavra, o USE produz representações vetoriais para sentenças inteiras, capturando o significado global e as relações contextuais entre palavras. Essas representações, com dimensões de **512** elementos, são úteis em diversas tarefas, como análise de similaridade semântica, busca de textos e classificação.

O modelo está disponível em variantes otimizadas para diferentes aplicações:
- Uma versão treinada em TensorFlow para uso em servidores de produção;
- Uma versão compacta e eficiente, adequada para dispositivos móveis e navegadores.

Além disso, o USE suporta entradas em várias línguas e utiliza redes neurais profundas baseadas em mecanismos de atenção e aprendizado multitarefa, tornando-o uma ferramenta poderosa para processar textos de forma semântica e contextualizada.

### Faz o embedding das colunas => coluna output
- **Texto** => `USE_Embeddings1`
- **TextoTratadoSemNomesLemmatized** => `USE_Embeddings4`

**OBS**: Utiliza o arquivo **./saida/03_3_DOUSecao02_portarias_mgi_tratado_NER_lematizado.parquet**, caso não exista, executar o **03_3_Tratamento_Lematizar.ipynb**

## Bibliotecas

In [1]:
# Biblioteca para manipulação e análise de dados
import pandas as pd

# Bibliotecas para aprendizado de máquina e processamento de texto
import tensorflow_hub as hub  # Para carregar modelos do TensorFlow Hub
import numpy as np  # Para operações numéricas e manipulação de arrays

# Biblioteca para medir o tempo de execução
import time

In [2]:
douItem = 2
secao = f'Secao0{douItem}'

# Lê o arquivo Parquet
df_portarias_mgi = pd.read_parquet(f'./saida/03_3_DOU{secao}_portarias_mgi_tratado_NER_lematizado.parquet', engine='pyarrow')  # ou engine='fastparquet'

# Exibe as primeiras linhas do DataFrame
df_portarias_mgi.shape


(19258, 39)

In [3]:
df_portarias_mgi.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,Texto,Total_palavras,TextoTratado,TextoTratado_TotalPalavras,Texto_sem_nomes,Nomes,TextoTratadoSemNomes,TextoTratadoSemNomes_TotalPalavras,TextoTratadoSemNomesLemmatized,TextoTratadoSemNomesLemmatized_TotalPalavras
0,30266027,PORTARIA DE PESSOAL 529,9368436,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 529, DE 17...",358,"sgp/sedgg/me nº 529, 17 janeiro 2023 secretari...",231,None,"sergio eduardo arbulu, anderson pereira silvy,...","sgp / sedgg / me nº 529 , 17 janeiro 2023 secr...",268,sgp Sedgg nº 529 17 janeiro 2023 secretario ge...,182
1,30264410,ATO PORTARIA 655,9368530,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA SGP/SEDGG/ME Nº 655, DE 23 DE JANEIRO...",245,"portaria sgp/sedgg/me nº 655, 23 janeiro 2023 ...",158,None,"eliane araujo franklin, eduardo arbulu","portaria sgp / sedgg / me nº 655 , 23 janeiro ...",189,portar sgp Sedgg nº 655 23 janeiro 2023 secret...,127
2,30265009,ATO PORTARIA DE PESSOAL 461,9368536,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 461, DE 16...",251,"sgp/sedgg/me nº 461, 16 janeiro 2023 secretari...",160,None,eduardo arbulu,"sgp / sedgg / me nº 461 , 16 janeiro 2023 secr...",197,sgp Sedgg nº 461 16 janeiro 2023 secretario ge...,132
3,30265678,ATO PORTARIA DE PESSOAL SGP 606,9368541,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 606, DE 19...",251,"sgp/sedgg/me nº 606, 19 janeiro 2023 secretari...",158,None,eduardo arbulu,"sgp / sedgg / me nº 606 , 19 janeiro 2023 secr...",187,sgp Sedgg nº 606 19 janeiro 2023 secretario ge...,134
4,30266030,ATO PORTARIA DE PESSOAL SGP 517,9368573,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 517, DE 17...",243,"sgp/sedgg/me nº 517, 17 janeiro 2023 secretari...",158,None,"roberto jorge ferreira, termino exercicio, edu...","sgp / sedgg / me nº 517 , 17 janeiro 2023 secr...",177,sgp Sedgg nº 517 17 janeiro 2023 secretario ge...,122


In [4]:
# Carregar o modelo Universal Sentence Encoder (USE)
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Função para criar embeddings
def create_embeddings(texts):
    """
    Gera embeddings para uma lista de textos usando o Universal Sentence Encoder (USE).
    
    Args:
        texts (list): Lista de textos para os quais os embeddings serão gerados.
    
    Returns:
        np.ndarray: Matriz de embeddings, onde cada linha é o embedding de um texto.
    """
    return use_model(texts).numpy()

# Garantir que as colunas de texto não tenham valores nulos
columns_to_process = ['Texto', 'TextoTratadoSemNomesLemmatized']

for column in columns_to_process:
    df_portarias_mgi[column] = df_portarias_mgi[column].fillna("")

# Função para processar todo o DataFrame linha a linha
def process_all_rows_line_by_line(df, columns, output_columns):
    """
    Processa todas as linhas do DataFrame, uma linha por vez.

    Args:
        df (pd.DataFrame): DataFrame contendo as colunas de texto para processar.
        columns (list): Lista de nomes das colunas de texto para gerar embeddings.
        output_columns (list): Lista de nomes das colunas de saída para armazenar os embeddings.

    Returns:
        pd.DataFrame: DataFrame com as colunas de embeddings adicionadas.
    """
    total_start_time = time.time()  # Início do tempo total

    for col, out_col in zip(columns, output_columns):
         # Garantir que a coluna de saída exista
        if out_col not in df.columns:
            df[out_col] = None

        # Processar linha por linha
        for idx in range(len(df)):
            text = df[col].iloc[idx]

            # Gerar embedding para a linha
            embedding = create_embeddings([text])[0]

            # Armazenar o embedding na coluna de saída
            df.at[idx, out_col] = embedding

            # Exibir progresso a cada 200 linhas
            if (idx + 1) % 1000 == 0:
                print(f"  Processadas {idx + 1} linhas da coluna {col}...")

    # Fim do tempo total
    total_time = time.time() - total_start_time
    print(f"\nTempo total de processamento: {total_time:.2f} segundos.")

    return df

# Configuração de colunas de entrada e saída
output_columns = ['USE_Embeddings1', 'USE_Embeddings4']

# Processar todo o DataFrame, linha a linha
print("Início do processamento de embeddings para todas as linhas...")
df_portarias_mgi = process_all_rows_line_by_line(
    df_portarias_mgi,
    columns_to_process,
    output_columns
)

print("Processamento concluído!")

Início do processamento de embeddings para todas as linhas...
  Processadas 1000 linhas da coluna Texto...
  Processadas 2000 linhas da coluna Texto...
  Processadas 3000 linhas da coluna Texto...
  Processadas 4000 linhas da coluna Texto...
  Processadas 5000 linhas da coluna Texto...
  Processadas 6000 linhas da coluna Texto...
  Processadas 7000 linhas da coluna Texto...
  Processadas 8000 linhas da coluna Texto...
  Processadas 9000 linhas da coluna Texto...
  Processadas 10000 linhas da coluna Texto...
  Processadas 11000 linhas da coluna Texto...
  Processadas 12000 linhas da coluna Texto...
  Processadas 13000 linhas da coluna Texto...
  Processadas 14000 linhas da coluna Texto...
  Processadas 15000 linhas da coluna Texto...
  Processadas 16000 linhas da coluna Texto...
  Processadas 17000 linhas da coluna Texto...
  Processadas 18000 linhas da coluna Texto...
  Processadas 19000 linhas da coluna Texto...
  Processadas 1000 linhas da coluna TextoTratadoSemNomesLemmatized...
  P

In [5]:
df_portarias_mgi.shape

(19258, 41)

In [6]:
df_portarias_mgi.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,TextoTratado,TextoTratado_TotalPalavras,Texto_sem_nomes,Nomes,TextoTratadoSemNomes,TextoTratadoSemNomes_TotalPalavras,TextoTratadoSemNomesLemmatized,TextoTratadoSemNomesLemmatized_TotalPalavras,USE_Embeddings1,USE_Embeddings4
0,30266027,PORTARIA DE PESSOAL 529,9368436,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"sgp/sedgg/me nº 529, 17 janeiro 2023 secretari...",231,None,"sergio eduardo arbulu, anderson pereira silvy,...","sgp / sedgg / me nº 529 , 17 janeiro 2023 secr...",268,sgp Sedgg nº 529 17 janeiro 2023 secretario ge...,182,"[-0.050729595, -0.01408702, 0.053643852, 0.055...","[-0.04014247, -0.06005167, -0.046900157, 0.053..."
1,30264410,ATO PORTARIA 655,9368530,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"portaria sgp/sedgg/me nº 655, 23 janeiro 2023 ...",158,None,"eliane araujo franklin, eduardo arbulu","portaria sgp / sedgg / me nº 655 , 23 janeiro ...",189,portar sgp Sedgg nº 655 23 janeiro 2023 secret...,127,"[-0.048080508, 0.013370708, 0.05068045, 0.0507...","[-0.030576829, -0.0599243, -0.040472377, 0.061..."
2,30265009,ATO PORTARIA DE PESSOAL 461,9368536,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"sgp/sedgg/me nº 461, 16 janeiro 2023 secretari...",160,None,eduardo arbulu,"sgp / sedgg / me nº 461 , 16 janeiro 2023 secr...",197,sgp Sedgg nº 461 16 janeiro 2023 secretario ge...,132,"[-0.043791097, 0.013420352, 0.05320274, 0.0461...","[-0.019732608, -0.061047755, -0.034037452, 0.0..."
3,30265678,ATO PORTARIA DE PESSOAL SGP 606,9368541,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"sgp/sedgg/me nº 606, 19 janeiro 2023 secretari...",158,None,eduardo arbulu,"sgp / sedgg / me nº 606 , 19 janeiro 2023 secr...",187,sgp Sedgg nº 606 19 janeiro 2023 secretario ge...,134,"[-0.045023933, -0.0064749066, 0.03732383, 0.04...","[-0.047755465, -0.06160219, -0.036089916, 0.04..."
4,30266030,ATO PORTARIA DE PESSOAL SGP 517,9368573,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,"sgp/sedgg/me nº 517, 17 janeiro 2023 secretari...",158,None,"roberto jorge ferreira, termino exercicio, edu...","sgp / sedgg / me nº 517 , 17 janeiro 2023 secr...",177,sgp Sedgg nº 517 17 janeiro 2023 secretario ge...,122,"[-0.05169902, -0.025740517, 0.04578067, 0.0534...","[-0.049283903, -0.063080825, -0.030478371, 0.0..."


In [7]:
# Salva saída parcial o arquivo completo com novas colunas
df_portarias_mgi.to_parquet(f'./saida/04_1_DOU{secao}_portarias_mgi_tratado_NER_lematizado_embeddingsUSE.parquet', engine='pyarrow', index=False)